In [10]:
import pandas as pd

# Load your dataset and Scopus index
dataset_path = 'Subject Area Mapping/Cleaned_CSV_Data.csv'  # Replace with the path to your dataset
scopus_index_path = 'Subject Area Mapping/ScopusIndex.csv'  # Replace with the path to your Scopus index file

data = pd.read_csv(dataset_path)
scopus_index = pd.read_csv(scopus_index_path)

# Create a new 'Category' column based on the top-level subject areas
scopus_index['Category'] = scopus_index[['Top level:\n\nLife Sciences', 'Top level:\n\nSocial Sciences',
                                         'Top level:\n\nPhysical Sciences', 'Top level:\n\nHealth Sciences']].bfill(axis=1).iloc[:, 0]

# Select subcategories that will be placed after column Z
subcategory_columns = ['1100\nAgricultural and Biological Sciences', '1200\nArts and Humanities',
                       '1300\nBiochemistry, Genetics and Molecular Biology', '1400\nBusiness, Management and Accounting',
                       '1700\nComputer Science', '2200\nEngineering', '2600\nMathematics']

# Merge the dataset with the Scopus index based on journal titles
merged_data = pd.merge(data, scopus_index[['Source Title', 'Print-ISSN', 'E-ISSN', 'Category'] + subcategory_columns], 
                       left_on='journal', right_on='Source Title', how='left')

# Drop unnecessary columns
merged_data = merged_data.drop(columns=['Source Title'])

# Create new columns for subcategories (Subcategory 1, Subcategory 2, etc.)
subcategories_list = []

for i in range(len(merged_data)):
    subcategories = [merged_data[col].iloc[i] for col in subcategory_columns if pd.notnull(merged_data[col].iloc[i])]
    subcategories_list.append(subcategories)

# Create Subcategory columns dynamically
for i in range(1, max(map(len, subcategories_list)) + 1):
    merged_data[f'Subcategory {i}'] = [subcategories[i - 1] if len(subcategories) >= i else None for subcategories in subcategories_list]

# Drop the original subcategory columns
merged_data = merged_data.drop(columns=subcategory_columns)

# Save the merged result to a new file if needed
merged_data.to_csv('C:/Users/shyla/Downloads/formatted_journals_with_subcategories.csv', index=False)

# Display the first few rows of the merged data to confirm the matching
print(merged_data.head())


                                               title  \
0  3D Printing for Cultural Heritage: Preservatio...   
1  Design for assistive technology oriented to de...   
2  Uncovering Challenges and Opportunities for 3D...   
3                                    Coming to Grips   
4  Interdisciplinary Contributions in the Design ...   

                                             journal  \
0                  Lecture Notes in Computer Science   
1  Journal of the Brazilian Society of Mechanical...   
2  Proceedings of the 18th International ACM SIGA...   
3                              Steamboat Connections   
4                                        TEM Journal   

                                             authors  year  \
0                  Neumüller, Reichinger, Rist, Kern  2014   
1                                   Santos, Silveira  2021   
2  McDonald, Comrie, Buehler, Carter, Dubin, Gord...  2016   
3                                                NaN  2000   
4      González-